# How To Clean Data in Excel Using Python?

This workbook expores five common Excel data cleaning techniques and demonstrates how to implement them in Python.

## The Scenario:
Let’s expand on the example from the previous article.  The orders database contains more information than previously analysed. 

The database also contains:
1.	The name of the client contact placing the order.
2.	The address of the client

In a separate Company Details table, we also have:

3.	The customer names (and not just a customer number)
4.	The sales representative for each customer.

The increased availability of data means that we can produce more useful data analytics for the business, and they have expanded their ask as a result.

We have been asked to:

1.	Provide a breakdown of total order numbers (volume and value) by: 
    1.	Customer Name (Not number)
    2.  Sales Rep Name
    3.	Country
2.	Isolate the name of the person raising the order and each individual element of the address as separate variables so that shipping labels can be printed.

The following seven steps clean the provided data and enable this analysis to be completed


In [21]:
# Import packages required
import pandas as pd

In [22]:
# Import messy_data from Excel
messy_data = pd.read_excel('messy_order_data.xlsx')

# Output head of mess_data to the screen
messy_data.head()

,Order Number,Customer,Units Ordered,Price Per Unit (£s),Order Email,Client Address
0,ORDER-06,CUSTOMER-1,100,£---289,DANA.NGUYEN@CUSTOMER-1.COM,PSC 4115\nBox 7815\nAPO\nENGLAND\nAA 41945
1,ORDER-11,CUSTOMER-1,138,£---896,DANA.NGUYEN@CUSTOMER-1.COM,PSC 4115\nBox 7815\nAPO\nENGLAND\nAA 41945
2,ORDER-12,CUSTOMER-1,136,£---905,DANA.NGUYEN@CUSTOMER-1.COM,PSC 4115\nBox 7815\nAPO\nENGLAND\nAA 41945
3,ORDER-15,CUSTOMER-1,79,£---325,CHERYL.BRADLEY@CUSTOMER-1.COM,PSC 4115\nBox 7815\nAPO\nENGLAND\nAA 41945
4,ORDER-16,CUSTOMER-1,132,"£---1,058",DANA.NGUYEN@CUSTOMER-1.COM,PSC 4115\nBox 7815\nAPO\nENGLAND\nAA 41945


## Step 1: Cleaning Price Per Unit Value

See last article for details regarding this.  This is now implemented in a more pythonic way using REGEX, which will be covered in future articles.

In [23]:
# Implement cleaning activity from previous article
messy_data['Price Per Unit (£s)'] = messy_data['Price Per Unit (£s)'].str.replace('£---|,','', regex=True).astype(float)

## Step 2: Calculating the Total Order Value

Pandas provides us with a simple way to apply formulars to data series that have the same shape (in Excel speak, columns of data that have the same number of rows populated).  We can simply apply the mathematical operations we want to directly to the columns. As the number of elements in each series is the same, Pandas knows to apply the function to corresponding values of the same index.

This means we can calculate the total order value per line by implementing the following code

In [24]:
# Calculate Total Order Value
messy_data['Total Order Value (£s)'] = messy_data['Units Ordered'] *  messy_data['Price Per Unit (£s)']

# Output head of mess_data to the screen (limited to the variables that have been used in calcualtion)
messy_data[['Units Ordered','Price Per Unit (£s)','Total Order Value (£s)']].head()

,Units Ordered,Price Per Unit (£s),Total Order Value (£s)
0,100,289.0,28900.0
1,138,896.0,123648.0
2,136,905.0,123080.0
3,79,325.0,25675.0
4,132,1058.0,139656.0


## Step 3: Splitting Client Address Column into Multiple Variables

Noting that the data is in the same format for all values, (Street / Box Number / Country / Mail Code), the following code will split the data and return individual variables for each element.

As we have duplicated the data from the Client Address in the new individual variables, we can now drop the original data from the messy_data dataframe.  The last line of code does this for us and the variable ```axis = 1```, tells the ```drop()``` function we are looking to drop a column.

In [25]:
# Define a list of new variable names for the seperated data from the Client Address
address_details = ['Street','Mail Box','County','Country','Mail Code']

# Split Client Data in to seperate columns and assign to the columns named in the list address_details in the messy_data dataframe
messy_data[address_details] = messy_data['Client Address'].str.split('\n',expand=True)

# Remove the original Client Address variable from the data
messy_data = messy_data.drop(['Client Address'],axis=1)

# Output head of mess_data to the screen
messy_data.head()

,Order Number,Customer,Units Ordered,Price Per Unit (£s),Order Email,Total Order Value (£s),Street,Mail Box,County,Country,Mail Code
0,ORDER-06,CUSTOMER-1,100,289.0,DANA.NGUYEN@CUSTOMER-1.COM,28900.0,PSC 4115,Box 7815,APO,ENGLAND,AA 41945
1,ORDER-11,CUSTOMER-1,138,896.0,DANA.NGUYEN@CUSTOMER-1.COM,123648.0,PSC 4115,Box 7815,APO,ENGLAND,AA 41945
2,ORDER-12,CUSTOMER-1,136,905.0,DANA.NGUYEN@CUSTOMER-1.COM,123080.0,PSC 4115,Box 7815,APO,ENGLAND,AA 41945
3,ORDER-15,CUSTOMER-1,79,325.0,CHERYL.BRADLEY@CUSTOMER-1.COM,25675.0,PSC 4115,Box 7815,APO,ENGLAND,AA 41945
4,ORDER-16,CUSTOMER-1,132,1058.0,DANA.NGUYEN@CUSTOMER-1.COM,139656.0,PSC 4115,Box 7815,APO,ENGLAND,AA 41945


## Step 4: Extract the Name of The Ordering Individual

Using ‘@’ as a delimiter, we can adapt the technique applied in step 3 to extract the first and surnames from the email address. 

Before we carry out the data cleansing activity, lets explore the output the function when '@' is used as a delimiter

In [26]:
# Apply str.split to Order Email, using '@' as a delimiter (return head only)
messy_data['Order Email'].str.split('@',expand=True).head()

,0,1
0,DANA.NGUYEN,CUSTOMER-1.COM
1,DANA.NGUYEN,CUSTOMER-1.COM
2,DANA.NGUYEN,CUSTOMER-1.COM
3,CHERYL.BRADLEY,CUSTOMER-1.COM
4,DANA.NGUYEN,CUSTOMER-1.COM


We can see that this has returned two columns of data.  For our analysis, we only require the first of these, the zeroth indexed column.  We can isolate this by indexing with [0]

The following code implements the required cleansing step and replaces the . with a space to result in a corectly formatted name.

In [27]:
# Extract the first name and last name from the Order Email and add to messy_data
messy_data['Order Name'] = messy_data['Order Email'].str.split('@',expand=True)[0].str.replace('.',' ')

messy_data.head()

,Order Number,Customer,Units Ordered,Price Per Unit (£s),Order Email,Total Order Value (£s),Street,Mail Box,County,Country,Mail Code,Order Name
0,ORDER-06,CUSTOMER-1,100,289.0,DANA.NGUYEN@CUSTOMER-1.COM,28900.0,PSC 4115,Box 7815,APO,ENGLAND,AA 41945,DANA NGUYEN
1,ORDER-11,CUSTOMER-1,138,896.0,DANA.NGUYEN@CUSTOMER-1.COM,123648.0,PSC 4115,Box 7815,APO,ENGLAND,AA 41945,DANA NGUYEN
2,ORDER-12,CUSTOMER-1,136,905.0,DANA.NGUYEN@CUSTOMER-1.COM,123080.0,PSC 4115,Box 7815,APO,ENGLAND,AA 41945,DANA NGUYEN
3,ORDER-15,CUSTOMER-1,79,325.0,CHERYL.BRADLEY@CUSTOMER-1.COM,25675.0,PSC 4115,Box 7815,APO,ENGLAND,AA 41945,CHERYL BRADLEY
4,ORDER-16,CUSTOMER-1,132,1058.0,DANA.NGUYEN@CUSTOMER-1.COM,139656.0,PSC 4115,Box 7815,APO,ENGLAND,AA 41945,DANA NGUYEN


## Step 5: Correcting the Capitalization of Text

First, lets inspect the Order Name and Country Variables

In [28]:
messy_data[['Order Name','Country']].head()

,Order Name,Country
0,DANA NGUYEN,ENGLAND
1,DANA NGUYEN,ENGLAND
2,DANA NGUYEN,ENGLAND
3,CHERYL BRADLEY,ENGLAND
4,DANA NGUYEN,ENGLAND


We can see that there are two variables which are fully capitalized.  As they contain proper nouns, we want to capitalize the first letter of them only

In [29]:
# Update the Country variable to be a title case (i.e. capitalise the first letter of each word)
messy_data['Country'] = messy_data['Country'].str.title()

# Update the Order Name variable to be a title case
messy_data['Order Name'] = messy_data['Order Name'].str.title()

# Output the head of the messy_data data fram to the screen
messy_data.head()

,Order Number,Customer,Units Ordered,Price Per Unit (£s),Order Email,Total Order Value (£s),Street,Mail Box,County,Country,Mail Code,Order Name
0,ORDER-06,CUSTOMER-1,100,289.0,DANA.NGUYEN@CUSTOMER-1.COM,28900.0,PSC 4115,Box 7815,APO,England,AA 41945,Dana Nguyen
1,ORDER-11,CUSTOMER-1,138,896.0,DANA.NGUYEN@CUSTOMER-1.COM,123648.0,PSC 4115,Box 7815,APO,England,AA 41945,Dana Nguyen
2,ORDER-12,CUSTOMER-1,136,905.0,DANA.NGUYEN@CUSTOMER-1.COM,123080.0,PSC 4115,Box 7815,APO,England,AA 41945,Dana Nguyen
3,ORDER-15,CUSTOMER-1,79,325.0,CHERYL.BRADLEY@CUSTOMER-1.COM,25675.0,PSC 4115,Box 7815,APO,England,AA 41945,Cheryl Bradley
4,ORDER-16,CUSTOMER-1,132,1058.0,DANA.NGUYEN@CUSTOMER-1.COM,139656.0,PSC 4115,Box 7815,APO,England,AA 41945,Dana Nguyen


## Step 6: Identifying and Removing Duplicate Rows of Data

By importing and visually inspecting the compay_data dataframe, we can see that there are duplicate entries in rows 3 and 4

In [30]:
# Read in the company data to use in the look up
company_data = pd.read_excel('company_details.xlsx')

# Print duplicate data and shape of data to the screen
print('Shape of company_data dataframe:',company_data.shape)
company_data

Shape of company_data dataframe: (5, 3)


,Customer,Customer Name,Sales Rep
0,CUSTOMER-1,"Perez, Torres and Moore",Tim Brady
1,CUSTOMER-2,Alvarez Group,Joshua Dyer
2,CUSTOMER-3,Valencia-Sanchez,Carrie Brady
3,CUSTOMER-4,"Ryan, Randolph and Armstrong",Carrie Brady
4,CUSTOMER-4,"Ryan, Randolph and Armstrong",Carrie Brady


The Pandas function ```drop_duplicates()``` provides a quick way to remove these from our dataframe.  The function is quite versatile and can be applied in many ways.  

In [31]:
# Remove duplicate rows from the company_data dataframe 
company_data = company_data.drop_duplicates()

# Print deduplicated data and shape of data to the screen
print('Shape of company_data dataframe:',company_data.shape)
company_data

Shape of company_data dataframe: (4, 3)


,Customer,Customer Name,Sales Rep
0,CUSTOMER-1,"Perez, Torres and Moore",Tim Brady
1,CUSTOMER-2,Alvarez Group,Joshua Dyer
2,CUSTOMER-3,Valencia-Sanchez,Carrie Brady
3,CUSTOMER-4,"Ryan, Randolph and Armstrong",Carrie Brady


## Step 7: Looking Up Values

Pandas alows us to look up data with the the ```merge()``` function. The behaviour of this function is based on a database join between two tables (the ‘Left table and the ‘Right table’) and in specific circumstances, will behave in the same way as a VLOOKUP in Excel.

In our example, we have our left table (the messy_data) and we want to look up the values from the right table (the company_data) via a match in values of the Customer ID (the ON value for the join)
If we have a many-to-one relationship between the left and right tables (that is many instances of the look-up ‘on’ variable values in the left table, but only one instance in the right table values) then the default merge will behave in the same way as a VLOOKUP.

If there are duplicates in the right table, it will permute and return all possible permutations of lookups.  This will introduce duplicates into the resulting dataframe, something that depending on the ask may or may not be desirable.  We need to be careful with the application of the merge function and check the shape of the dataframes both before and after.


In [32]:
# Print the shape of the meesy_data dataframe to the screen prior to merge
print('Shape of messy_data prior to merge:', messy_data.shape)

Shape of messy_data prior to merge: (50, 12)


In [33]:
messy_data.merge(company_data, on = 'Customer').shape

(50, 14)

In [34]:
# Print the shape of the meesy_data dataframe to the screen prior to merge
print('Shape of messy_data prior to merge:', messy_data.shape)

# Merge data sets and lookup values in the messy_data and customer_data via an Inner Join on the 'Customer' variable
clean_data = messy_data.merge(company_data, on = 'Customer')

# Print the shape of the meesy_data dataframe to the screen prior to merge
print('Shape of clean_data post merge:', clean_data.shape)

# Print head of clean_data to the screen
clean_data.head()

Shape of messy_data prior to merge: (50, 12)
Shape of clean_data post merge: (50, 14)


,Order Number,Customer,Units Ordered,Price Per Unit (£s),Order Email,Total Order Value (£s),Street,Mail Box,County,Country,Mail Code,Order Name,Customer Name,Sales Rep
0,ORDER-06,CUSTOMER-1,100,289.0,DANA.NGUYEN@CUSTOMER-1.COM,28900.0,PSC 4115,Box 7815,APO,England,AA 41945,Dana Nguyen,"Perez, Torres and Moore",Tim Brady
1,ORDER-11,CUSTOMER-1,138,896.0,DANA.NGUYEN@CUSTOMER-1.COM,123648.0,PSC 4115,Box 7815,APO,England,AA 41945,Dana Nguyen,"Perez, Torres and Moore",Tim Brady
2,ORDER-12,CUSTOMER-1,136,905.0,DANA.NGUYEN@CUSTOMER-1.COM,123080.0,PSC 4115,Box 7815,APO,England,AA 41945,Dana Nguyen,"Perez, Torres and Moore",Tim Brady
3,ORDER-15,CUSTOMER-1,79,325.0,CHERYL.BRADLEY@CUSTOMER-1.COM,25675.0,PSC 4115,Box 7815,APO,England,AA 41945,Cheryl Bradley,"Perez, Torres and Moore",Tim Brady
4,ORDER-16,CUSTOMER-1,132,1058.0,DANA.NGUYEN@CUSTOMER-1.COM,139656.0,PSC 4115,Box 7815,APO,England,AA 41945,Dana Nguyen,"Perez, Torres and Moore",Tim Brady


## Export back to Excel

The code below exports the data back to Excel.  This applies the column ordering set out in the list ```column_ordering```.


In [35]:
# Define order of columns for outputed data
column_ordering = ['Order Number', 'Customer', 'Customer Name','Sales Rep','Units Ordered',
                   'Price Per Unit (£s)','Total Order Value (£s)', 'Order Name', 
                   'Order Email', 'Street', 'Mail Box', 'County', 'Country', 'Mail Code']

# Export clean data to excel file for analysis and processing
clean_data[column_ordering].to_excel('Clean Order Data.xlsx',index = False)

## Example of Coercion Error in Python

The following code provides an example of a coercion error in Python.

In [36]:
# Initiate example data frame to demonstrate coercion errors
messy_values = pd.DataFrame({'Price Per Order (£s)':['£---1,054.63','£---1,629.57','£---1,606.67','£---1,201.03']})

# Output messy_values to screen
messy_values['Price Per Order (£s)']

0    £---1,054.63
1    £---1,629.57
2    £---1,606.67
3    £---1,201.03
Name: Price Per Order (£s), dtype: object

In [37]:
# If we remove the unecessary text and leave a text value contianing only numeric values, we can coerce to a float
messy_values['Price Per Order (£s)'].str.replace('£---|,','', regex=True).astype(float)

0    1054.63
1    1629.57
2    1606.67
3    1201.03
Name: Price Per Order (£s), dtype: float64

In [38]:
# If we don't remove the unecessary characters, we cant coerce the values.
messy_values['Price Per Order (£s)'].astype(float)

ValueError: could not convert string to float: '£---1,054.63'